In [1]:
!pip install imbalanced-learn
!pip install --upgrade numexpr

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, matthews_corrcoef, balanced_accuracy_score
from imblearn.over_sampling import SMOTE

# Load the dataset
file_path = 'End_dataframe.csv' 
data = pd.read_csv(file_path)

# Features and target
X = data[['Gender', 'Prior_Donation', 'Lcheek_max', 'Rcheek_max', 'nose_max', 'chin_max', 'below_nose_max', 'HRV_minmax']]
y = data['VVR_Encoded']  

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Apply SMOTE to balance the classes in the training set
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Define the Random Forest model and parameter grid for Grid Search
rf = RandomForestClassifier(random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'ccp_alpha': [0.001, 0.01, 0.1, 1]
}

# Grid Search with cross-validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_resampled.drop(columns='Gender'), y_train_resampled)

# Print the best parameters and the best score
print("\nOptimal Parameters from Grid Search:")
print(grid_search.best_params_)
print(f"Best Cross-Validation Accuracy: {grid_search.best_score_:.2f}")

# Best model from Grid Search
best_rf = grid_search.best_estimator_

# Make predictions on the test set and training set
y_pred_test = best_rf.predict(X_test.drop(columns='Gender'))
y_pred_train = best_rf.predict(X_train_resampled.drop(columns='Gender'))

# Function to compute per-class balanced accuracy
def per_class_balanced_accuracy(cm):
    per_class = cm.diagonal() / cm.sum(axis=1)
    return per_class

# Test set evaluation
test_cm = confusion_matrix(y_test, y_pred_test)
test_balanced_acc = balanced_accuracy_score(y_test, y_pred_test)
test_per_class_balanced_acc = per_class_balanced_accuracy(test_cm)

print("\nTest Set Performance:")
print(f"Balanced Accuracy (Test Set): {test_balanced_acc:.2f}")
for i, acc in enumerate(test_per_class_balanced_acc):
    print(f"Class {i} Balanced Accuracy: {acc:.2f}")
print("\nConfusion Matrix (Test Set):")
print(test_cm)
print("\nClassification Report (Test Set):")
print(classification_report(y_test, y_pred_test))

# Training set evaluation
train_cm = confusion_matrix(y_train_resampled, y_pred_train)
train_balanced_acc = balanced_accuracy_score(y_train_resampled, y_pred_train)
train_per_class_balanced_acc = per_class_balanced_accuracy(train_cm)

print("\nTraining Set Performance:")
print(f"Balanced Accuracy (Training Set): {train_balanced_acc:.2f}")
for i, acc in enumerate(train_per_class_balanced_acc):
    print(f"Class {i} Balanced Accuracy: {acc:.2f}")
print("\nConfusion Matrix (Training Set):")
print(train_cm)
print("\nClassification Report (Training Set):")
print(classification_report(y_train_resampled, y_pred_train))

# Gender-wise analysis for test set
class_labels = {0: 'Low_VVR', 1: 'High_VVR'}
gender_labels = {1: 'Male', 2: 'Female'}

X_test['y_test'] = y_test  
X_test['y_pred'] = y_pred_test  

print("\nTest Set Performance by Gender:")
test_gender_groups = X_test.groupby('Gender')
for gender, group in test_gender_groups:
    gender_name = gender_labels.get(gender, gender)
    true_labels = group['y_test']
    predicted_labels = group['y_pred']
    print(f"\nGender: {gender_name}")
    print(confusion_matrix(true_labels, predicted_labels, labels=[0, 1]))
    print(classification_report(true_labels, predicted_labels, target_names=['Low_VVR', 'High_VVR']))
    print(f"MCC: {matthews_corrcoef(true_labels, predicted_labels):.2f}")

# Gender-wise analysis for training set
print("\nTraining Set Performance by Gender:")
X_train_resampled['y_train'] = y_train_resampled  # Add true labels
X_train_resampled['y_pred'] = y_pred_train  # Add predictions

train_gender_groups = X_train_resampled.groupby('Gender')
for gender, group in train_gender_groups:
    gender_name = gender_labels.get(gender, gender)
    true_labels = group['y_train']
    predicted_labels = group['y_pred']
    print(f"\nGender: {gender_name}")
    print(confusion_matrix(true_labels, predicted_labels, labels=[0, 1]))
    print(classification_report(true_labels, predicted_labels, target_names=['Low_VVR', 'High_VVR']))
    print(f"MCC: {matthews_corrcoef(true_labels, predicted_labels):.2f}")



Optimal Parameters from Grid Search:
{'ccp_alpha': 0.001, 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Best Cross-Validation Accuracy: 0.73

Test Set Performance:
Balanced Accuracy (Test Set): 0.55
Class 0 Balanced Accuracy: 0.68
Class 1 Balanced Accuracy: 0.41

Confusion Matrix (Test Set):
[[63 29]
 [22 15]]

Classification Report (Test Set):
              precision    recall  f1-score   support

           0       0.74      0.68      0.71        92
           1       0.34      0.41      0.37        37

    accuracy                           0.60       129
   macro avg       0.54      0.55      0.54       129
weighted avg       0.63      0.60      0.61       129


Training Set Performance:
Balanced Accuracy (Training Set): 0.99
Class 0 Balanced Accuracy: 1.00
Class 1 Balanced Accuracy: 0.99

Confusion Matrix (Training Set):
[[363   1]
 [  5 359]]

Classification Report (Training Set):
              precision    recall  f1-score   support

     